In [1]:
# Импорт библиотек
import pandas as pd
import os
import random
from math import pi
import torch
from torch.utils.data import random_split, Dataset, DataLoader, TensorDataset
from torch import nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [9]:
dfNormal1 = pd.read_parquet('../00. Resources/Datasets/First/Normal.A1.parquet')
#print(dfNormal1.head)
dfNormal3 = pd.read_parquet('../00. Resources/Datasets/First/Normal.A3.parquet')
#print(dfNormal3.head)
dfIrregular1 = pd.read_parquet('../00. Resources/Datasets/First/Irregular.A1.parquet')
#print(dfIrregular1.head)
dfIrregular3 = pd.read_parquet('../00. Resources/Datasets/First/Irregular.A3.parquet')
#print(dfIrregular3.head)
dfNoises1 = pd.read_parquet('../00. Resources/Datasets/First/Noises.A1.parquet')
#print(dfNoises1.head)

df = pd.concat([dfNormal1, dfNormal3, dfIrregular1, dfIrregular3, dfNoises1], ignore_index=True)

# df = pd.read_parquet('trash/dataset0.parquet')
df.reset_index(drop=True)
df['value'] = (df['value'] - df['value'].min()) / (df['value'].max() - df['value'].min())       # Нормализация данных

In [ ]:
class FirstEcgDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.segmentStarts = df.index[df.index % 200 == 0].tolist()
        self.segmentsCount = len(self.segmentStarts)
        self.cachedTensors = [self._get_segment_tensor(index) for index in range(self.segmentsCount)]
        self.cachedLabels = [self._get_label_tensor(index) for index in range(self.segmentsCount)]
    
    def _get_segment_tensor(self, index):
        segment = self._get_segment(index)
        segmentValues = segment['value']
        segmentTensor = torch.tensor(segmentValues.to_numpy(), dtype=torch.float).unsqueeze(1)
        return segmentTensor

    def _get_label_tensor(self, index):
        segment = self._get_segment(index)
        labelValues = segment['edge']
        labelTensor = torch.tensor(labelValues.to_numpy(), dtype=torch.float).unsqueeze(1)
        return labelTensor

    def _get_segment(self, index):
        if index + 1 < len(self.segmentStarts):
            return self.df.iloc[self.segmentStarts[index]:self.segmentStarts[index + 1]]
        else:
            return self.df.iloc[self.segmentStarts[index]:]

    # Часто вызываемые во врмемя вычислений функции (оптимизированы)

    def __len__(self):
        return self.segmentsCount

    def __getitem__(self, index):
        segmentTensor = self.cachedTensors[index]
        labelTensor = self.cachedLabels[index]
        return segmentTensor, labelTensor

dataset = FirstEcgDataset(df)
print(len(dataset))
trainDataset, valDataset = random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])

trainDataloader = torch.utils.data.DataLoader(trainDataset, batch_size=10, shuffle=True)
valDataloader = torch.utils.data.DataLoader(valDataset, batch_size=10, shuffle=False)

In [ ]:
class EcgLstmNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, bidirectional=True):
        super(EcgLstmNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)
        
        # Fully connected layer
        # Если LSTM двунаправленный, то умножаем hidden_size на 2 для конкатенации скрытых состояний из обоих направлений
        self.fc = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, 1)
        
        # Sigmoid layer для получения вероятности
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Инициализация скрытых состояний и состояний ячейки с нулями
        h0 = torch.zeros(self.num_layers * 2 if self.bidirectional else self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * 2 if self.bidirectional else self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Прямой проход через LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Преобразование выхода LSTM для предсказания вероятности
        out = self.fc(out)
        out = self.sigmoid(out)
        
        return out

# Параметры для создания модели
input_size = 1
hidden_size = 64
num_layers = 1

# Создание экземпляра модели
device = ("cuda")
model = EcgLstmNet(input_size, hidden_size, num_layers).to(device)
print(model)
# Определение функции потерь и оптимизатора
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

def train(trainDataloader, valDataloader, model, criterion, optimizer):
    # Обучение модели
    model.train()
    trainPbar = tqdm(trainDataloader, desc="Training")
    for inputs, targets in trainPbar:
        inputs, targets = inputs.to(device), targets.to(device)     # Передаём входные векторы видеокарте, чтобы вычисления производились на ней
        optimizer.zero_grad()
        outputs = model(inputs)     # Прогоняем через нашу нейронную сеть один батч
        loss = criterion(outputs, targets)      # Вычисляем функцию ошибок
        loss.backward()     # Вычисляем градиенты функции ошибок
        optimizer.step()        # Обновляем веса модели
        trainPbar.set_postfix({'Loss': f'{loss.item()}'})

    # Валидация модели
    model.eval() # Переводим модель в режим валидации
    valPbar = tqdm(valDataloader, desc="Validating")
    with torch.no_grad(): # Отключаем вычисление градиентов
        for inputs, targets in valPbar:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            val_loss = criterion(outputs, targets)
            valPbar.set_postfix({'Val_Loss': f'{val_loss.item()}'})
    
    trainPbar.close()
    valPbar.close()

In [ ]:
for epoch in range(30):
    print(f'Epoch: {epoch} ')
    train(trainDataloader, valDataloader, model, criterion, optimizer)

print('---------------------------------------------------------------------------------------------')
print('Done!')

In [8]:
# Сохранение модели
torch.save(model.state_dict(), '../00. Resources/Models/FirstStageModel.pt')